In [1]:
# import libraries 

from bs4 import BeautifulSoup
import requests
from time import sleep
from datetime import datetime
import csv
import re
import pandas as pd


In [7]:
# with open('Final_MAL_Scrap.csv', 'w', newline='', encoding='utf-8') as f:
#     fieldnames = ['Title', 'Type', 'Score', 'Score Count', 'Members', 'Rank', 'Popularity', 'Genres', 'Authors', 'Synopsis', 'Link', 'Date', 'Page']
#     thewriter = csv.DictWriter(f, fieldnames=fieldnames)
#     thewriter.writeheader()

# commented cuz ion wanna accidently snap the data ;)

In [ ]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"}
i = 0
#Read the existing data
df = pd.read_csv('Final_MAL_Scrap.csv')

for i in range(3800, 4000, 50): #(start, end, intervals)
    top_url = f"https://myanimelist.net/topanime.php?limit={i}"
    top_page = requests.get(top_url, headers=headers)
    top_soup = BeautifulSoup(top_page.content, "html.parser")
    # get link href link for each manga
    manga_list = top_soup.find_all("tr", class_="ranking-list")

    manga_links = []
    for manga in manga_list:
        manga_links.append(manga.find("a")["href"])

    # get manga information
    for manga_url in manga_links:

        # Check if the manga already exists in the CSV file
        if df['Link'].str.contains(manga_url).any():
            continue

        manga_page = requests.get(manga_url, headers=headers)
        manga_soup = BeautifulSoup(manga_page.content, "html.parser")

        manga_title = manga_soup.find("h1", class_="title-name").text
        manga_synopsis_element = manga_soup.find("p", itemprop="description")
        if manga_synopsis_element is not None:
            manga_synopsis = manga_synopsis_element.text.strip()
        else:
            manga_synopsis = ""

        #stats
        manga_score = manga_soup.find("div", class_="fl-l score").text.strip()
        manga_score_count = manga_soup.find("span", itemprop="ratingCount").text
        manga_members = manga_soup.find("span", class_="numbers members").text.strip().split(" ")[1]
        manga_rank = manga_soup.find("span", class_="numbers ranked").text.strip().split("#")[1]
        manga_popularity = manga_soup.find("span", class_="numbers popularity").text.strip().split("#")[1]

        #information
        manga_type = manga_soup.find("span", class_="information type").text.strip()
        manga_genres = manga_soup.find_all("span", itemprop="genre")
        genres = [genre.text.strip() for genre in manga_genres]
        authors = manga_soup.find("span", class_="information studio author")
        if authors is not None:
            authors = authors.text.strip()
        else:
            authors = ""

        #removig the commas and extra gap from the numbers
        manga_score_count = manga_score_count.strip()
        manga_members = manga_members.strip().replace(",", "")
        manga_rank = manga_rank.strip().replace(",", "")
        manga_popularity = manga_popularity.strip().replace(",", "")
        #removing extra gap from synopsis
        manga_synopsis = manga_synopsis.strip()

        #change numbers to int and score in float
        manga_score = float(manga_score)
        manga_score_count = int(manga_score_count)
        manga_members = int(manga_members)
        manga_rank = int(manga_rank)
        manga_popularity = int(manga_popularity)

        #joining generes in a string
        genres_str = ', '.join(genres)
        
        date_scrapped = datetime.now().strftime('%d-%m-%Y')
        page_type = "Top Anime"

        with open('Final_MAL_Scrap.csv', '+a', newline='', encoding='utf-8') as f:
            fieldnames = ['Title', 'Type', 'Score', 'Score Count', 'Members', 'Rank', 'Popularity', 'Genres', 'Authors', 'Synopsis', 'Link', 'Date', 'Page']
            thewriter = csv.DictWriter(f, fieldnames=fieldnames)
            thewriter.writerow({'Title': manga_title, 'Type': manga_type, 'Score': manga_score, 'Score Count': manga_score_count, 'Members': manga_members, 'Rank': manga_rank, 'Popularity': manga_popularity, 'Genres': genres_str, 'Authors': authors, 'Synopsis': manga_synopsis, 'Link': manga_url, 'Date': date_scrapped, 'Page': page_type})

        sleep(6)


In [3]:
# csv file is missing the head of first column add it as "Title"
# df = pd.read_csv('Final_MAL_Scrap.csv')
# df.columns = ['Title', 'Type', 'Score', 'Score Count', 'Members', 'Rank', 'Popularity', 'Genres', 'Authors', 'Synopsis', 'Link', 'Date', 'Page']
# df.to_csv('Final_MAL_Scrap.csv', index=False)
